<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session5_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 5 (Lexical Semmantics) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [1]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/

pip install python-crfsuite

Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 227 (delta 25), reused 12 (delta 12), pack-reused 190
Receiving objects: 100% (227/227), 264.31 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (115/115), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 13.6 MB/s eta 0:00:00


In [182]:
from scripts.models import StatisticalModels
import numpy as np
import nltk
import pandas as pd
nltk.download('treebank')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')
nltk.download('wordnet_ic')
from nltk.corpus import wordnet_ic
brown_ic = wordnet_ic.ic('ic-brown.dat')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


## TODO

 **Given the following (lemma, category) pairs:**   \

```
('the','DT'),('man','NN'),('swim','VB'),('with','PR'),('a','DT'),('girl','NN'),('and','CC'),
('a','DT'),('boy','NN'),('whilst','PR'),('the','DT'),('woman','NN'),('walk','VB')
```

1. For each pair, when possible, print their most frequent WordNet synset

2. For each pair of words, when possible, print their corresponding least common subsumer (LCS) and their similarity value, using the following functions:

  - Path Similarity
  - Leacock-Chodorow Similarity
  - Wu-Palmer Similarity
  - Lin Similarity

3. Normalize similarity values when necessary. What similarity seems better?

The code for the models is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/models.py


In [176]:
Lemmas= [('the','DT'),('man','NN'),('swim','VB'),('with','PR'),('a','DT'),('girl','NN'),('and','CC'),
('a','DT'),('boy','NN'),('whilst','PR'),('the','DT'),('woman','NN'),('walk','VB')]

In [177]:
#We apply a cleaning process to remove some possible duplicate values
Clean_pairs = list(set(Lemmas))
print(len(Lemmas)-len(Clean_pairs), 'lemma-category values are duplicates')

2 lemma-category values are duplicates


In [178]:
#Ex. 1 With the corresponding pairs cleaned and with the correct category id
# we can now review the most frequen synset

d = {'NN': 'n', 'NNS': 'n',
       'JJ': 'a', 'JJR': 'a', 'JJS': 'a',
       'VB': 'v', 'VBD': 'v', 'VBG': 'v', 'VBN': 'v', 'VBP': 'v', 'VBZ': 'v',
       'RB': 'r', 'RBR': 'r', 'RBS': 'r'}

def most_frequent (Lemmas):
  results={'accepted':[], 'empty':[]}
  for pair in range(len(Lemmas)):

    if Lemmas[pair][1] in d.keys():

      if len(wn.synsets(Lemmas[pair][0],d[Lemmas[pair][1]]))==0:
         results['empty'].append(Lemmas[pair])
      else:
        common= wn.synsets(Lemmas[pair][0],d[Lemmas[pair][1]])[0]
        tupple=(Lemmas[pair][0],common)
        results['accepted'].append(tupple)

    else:
      results['empty'].append(Lemmas[pair])

  return results

In [188]:
Lemmas_syn=most_frequent(Clean_pairs)
print(Lemmas_syn)

{'accepted': [('swim', Synset('swim.v.01')), ('walk', Synset('walk.v.01')), ('man', Synset('man.n.01')), ('girl', Synset('girl.n.01')), ('woman', Synset('woman.n.01')), ('boy', Synset('male_child.n.01'))], 'empty': [('with', 'PR'), ('and', 'CC'), ('whilst', 'PR'), ('a', 'DT'), ('the', 'DT')]}


In [185]:
#Now, we put all the lemma-category in all the possible pair combinations into the list "Pairs"
#Since there are 11 pairs, we should get 55 elements
Pairs = [(a, b) for idx, a in enumerate(Lemmas_syn['accepted']) for b in Lemmas_syn['accepted'][idx + 1:]]
Pairs

[(('swim', Synset('swim.v.01')), ('walk', Synset('walk.v.01'))),
 (('swim', Synset('swim.v.01')), ('man', Synset('man.n.01'))),
 (('swim', Synset('swim.v.01')), ('girl', Synset('girl.n.01'))),
 (('swim', Synset('swim.v.01')), ('woman', Synset('woman.n.01'))),
 (('swim', Synset('swim.v.01')), ('boy', Synset('male_child.n.01'))),
 (('walk', Synset('walk.v.01')), ('man', Synset('man.n.01'))),
 (('walk', Synset('walk.v.01')), ('girl', Synset('girl.n.01'))),
 (('walk', Synset('walk.v.01')), ('woman', Synset('woman.n.01'))),
 (('walk', Synset('walk.v.01')), ('boy', Synset('male_child.n.01'))),
 (('man', Synset('man.n.01')), ('girl', Synset('girl.n.01'))),
 (('man', Synset('man.n.01')), ('woman', Synset('woman.n.01'))),
 (('man', Synset('man.n.01')), ('boy', Synset('male_child.n.01'))),
 (('girl', Synset('girl.n.01')), ('woman', Synset('woman.n.01'))),
 (('girl', Synset('girl.n.01')), ('boy', Synset('male_child.n.01'))),
 (('woman', Synset('woman.n.01')), ('boy', Synset('male_child.n.01')))]

In [193]:
for pair in Pairs:
  pair[0][1].lowest_common_hypernyms(pair[1][1])
  pair[0][1].path_similarity(pair[1][1])
  pair[0][1].lowest_common_hypernyms(pair[1][1])
  print(test)

[Synset('travel.v.01')]
[]
[]
[]
[]
[]
[]
[]
[]
[Synset('adult.n.01')]
[Synset('adult.n.01')]
[Synset('male.n.02')]
[Synset('woman.n.01')]
[Synset('person.n.01')]
[Synset('person.n.01')]


In [189]:
#Similarities example

#def similar (pairs):
 # for pair in pairs:
  #  pairs
#wn.synset('dog.n.01')

wn.synset('cat.n.01')

#wn.synset('dog.n.01').lowest_common_hypernyms( wn.synset('cat.n.01'))

#wn.synset('dog.n.01').lowest_common_hypernyms( Synset('cat.n.01'))
#  [Synset('carnivore.n.01')]
#dog.path_similarity(cat)  👉  0.2
#dog.lch_similarity(cat)  👉  2.0281482472922856
#dog.wup_similarity(cat)  👉  0.8571428571428571
#dog.lin_similarity(cat,brown_ic)  👉  0.8768009843733973

Synset('cat.n.01')

In [ ]:
wn.synsets(Lemma[pair][0],d[Lemma[pair][1]])[0]

In [ ]:
.lowest_common_hypernyms('cat.n.01')

# Conclusion

